# Exploring the dataset

In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import kagglehub
from configurations import DATASET_PATH

## Downloading the dataset

In [7]:
local_path = kagglehub.dataset_download("asdasdasasdas/garbage-classification")
dataset_tmp_path = os.path.join(local_path, "Garbage classification", "Garbage classification")
classes = os.listdir(dataset_tmp_path)

dataset = {
    'image_name': [],
    'class_name': [],
    'image_path': [],
}
for class_name in classes:
    class_path = os.path.join(dataset_tmp_path, class_name)
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        dataset["class_name"].append(class_name)
        dataset["image_name"].append(image_name)
        dataset["image_path"].append(image_path)
        
dataset = pd.DataFrame.from_dict(dataset)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2527 entries, 0 to 2526
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  2527 non-null   object
 1   class_name  2527 non-null   object
 2   image_path  2527 non-null   object
dtypes: object(3)
memory usage: 59.4+ KB


In [8]:
dataset.to_csv(DATASET_PATH, sep=";", decimal=',', index=False)